### Importing Packages

In [1]:
import pandas as pd 
import numpy as np 
import seaborn as sns 

import os
from loguru import logger

import warnings
warnings.filterwarnings('ignore')
import gc

### Common functions

In [2]:
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
#         print(f'{col}:{col_type}')
        
        if col_type not in [object,'category']:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df

### Importing Data

##### Importing train base data

In [3]:
for dirname, _, filenames in os.walk('/kaggle/input/home-credit-credit-risk-model-stability/csv_files/train/'):
    for filename in filenames:
        logger.info(f"Filename is:{filename}")

2024-02-29 09:45:12.511 | INFO     | __main__:<module>:3 - Filename is:train_credit_bureau_a_1_3.csv
2024-02-29 09:45:12.513 | INFO     | __main__:<module>:3 - Filename is:train_static_cb_0.csv
2024-02-29 09:45:12.514 | INFO     | __main__:<module>:3 - Filename is:train_applprev_1_0.csv
2024-02-29 09:45:12.516 | INFO     | __main__:<module>:3 - Filename is:train_person_2.csv
2024-02-29 09:45:12.518 | INFO     | __main__:<module>:3 - Filename is:train_base.csv
2024-02-29 09:45:12.519 | INFO     | __main__:<module>:3 - Filename is:train_tax_registry_a_1.csv
2024-02-29 09:45:12.521 | INFO     | __main__:<module>:3 - Filename is:train_static_0_0.csv
2024-02-29 09:45:12.522 | INFO     | __main__:<module>:3 - Filename is:train_credit_bureau_a_1_0.csv
2024-02-29 09:45:12.523 | INFO     | __main__:<module>:3 - Filename is:train_applprev_2.csv
2024-02-29 09:45:12.524 | INFO     | __main__:<module>:3 - Filename is:train_credit_bureau_a_2_6.csv
2024-02-29 09:45:12.525 | INFO     | __main__:<modul

## Base data

In [4]:
df_base = pd.read_csv("/kaggle/input/home-credit-credit-risk-model-stability/csv_files/train/train_base.csv")

In [5]:
df_base.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1526659 entries, 0 to 1526658
Data columns (total 5 columns):
 #   Column         Non-Null Count    Dtype 
---  ------         --------------    ----- 
 0   case_id        1526659 non-null  int64 
 1   date_decision  1526659 non-null  object
 2   MONTH          1526659 non-null  int64 
 3   WEEK_NUM       1526659 non-null  int64 
 4   target         1526659 non-null  int64 
dtypes: int64(4), object(1)
memory usage: 58.2+ MB


In [6]:
logger.info(f"Number of unique records: {df_base['case_id'].nunique()}")

2024-02-29 09:45:14.421 | INFO     | __main__:<module>:1 - Number of unique records: 1526659


In [7]:
df_base = reduce_mem_usage(df_base)

Memory usage of dataframe is 58.24 MB
Memory usage after optimization is: 17.49 MB
Decreased by 70.0%


### Datasets with depth 0 

#### Static data

In [8]:
df_static_0 = pd.read_csv("/kaggle/input/home-credit-credit-risk-model-stability/csv_files/train/train_static_0_0.csv")
df_static_1 = pd.read_csv("/kaggle/input/home-credit-credit-risk-model-stability/csv_files/train/train_static_0_1.csv")

In [9]:
df_static_0 = reduce_mem_usage(df_static_0)
df_static_1 = reduce_mem_usage(df_static_1)

Memory usage of dataframe is 1279.85 MB
Memory usage after optimization is: 361.68 MB
Decreased by 71.7%
Memory usage of dataframe is 666.73 MB
Memory usage after optimization is: 190.29 MB
Decreased by 71.5%


In [10]:
logger.info(f"Number of unique records in 0: {df_static_0['case_id'].nunique()}")
logger.info(f"Number of unique records in 1: {df_static_1['case_id'].nunique()}")

2024-02-29 09:46:25.338 | INFO     | __main__:<module>:1 - Number of unique records in 0: 1003757
2024-02-29 09:46:25.354 | INFO     | __main__:<module>:2 - Number of unique records in 1: 522902


In [11]:
## Combining the datasets
df_static = pd.concat([df_static_0,df_static_1],ignore_index=True)

In [12]:
df_static = reduce_mem_usage(df_static)

Memory usage of dataframe is 768.74 MB
Memory usage after optimization is: 552.16 MB
Decreased by 28.2%


In [13]:
logger.info(f"Total records in static dataset: {df_static['case_id'].nunique()}")

2024-02-29 09:46:35.771 | INFO     | __main__:<module>:1 - Total records in static dataset: 1526659


#### Static CB

In [14]:
df_static_cb = pd.read_csv("/kaggle/input/home-credit-credit-risk-model-stability/csv_files/train/train_static_cb_0.csv")

In [15]:
logger.info(f"Number of unique records in 0: {df_static_cb['case_id'].nunique()}")

2024-02-29 09:46:48.991 | INFO     | __main__:<module>:1 - Number of unique records in 0: 1500476


In [16]:
df_static_cb = reduce_mem_usage(df_static_cb)
df_static_cb.info()

Memory usage of dataframe is 606.73 MB
Memory usage after optimization is: 159.58 MB
Decreased by 73.7%
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1500476 entries, 0 to 1500475
Data columns (total 53 columns):
 #   Column                   Non-Null Count    Dtype   
---  ------                   --------------    -----   
 0   case_id                  1500476 non-null  int32   
 1   assignmentdate_238D      136996 non-null   category
 2   assignmentdate_4527235D  114978 non-null   category
 3   assignmentdate_4955616D  71633 non-null    category
 4   birthdate_574D           607871 non-null   category
 5   contractssum_5085716L    157329 non-null   float32 
 6   dateofbirth_337D         1385691 non-null  category
 7   dateofbirth_342D         36500 non-null    category
 8   days120_123L             1385691 non-null  float16 
 9   days180_256L             1385691 non-null  float16 
 10  days30_165L              1385691 non-null  float16 
 11  days360_512L             1385691 non-

In [17]:
gc.collect()

0

##### Saving the new files 

In [18]:
!pwd

/kaggle/working


In [19]:
mkdir train

In [20]:
df_base.to_parquet('train/train_base.parquet')
df_static.to_parquet('train/train_static.parquet')
df_static_cb.to_parquet('train/train_static_cb.parquet')

### Datasets with Depth 1

In [21]:
df_applprev_1_0 = pd.read_csv("/kaggle/input/home-credit-credit-risk-model-stability/csv_files/train/train_applprev_1_0.csv")

df_applprev_1_1 = pd.read_csv("/kaggle/input/home-credit-credit-risk-model-stability/csv_files/train/train_applprev_1_1.csv")

df_applprev = pd.concat([df_applprev_1_0,df_applprev_1_1],ignore_index=True)

In [26]:
df_applprev = reduce_mem_usage(df_applprev)

Memory usage of dataframe is 2041.36 MB
Memory usage after optimization is: 592.79 MB
Decreased by 71.0%


In [27]:
df_applprev.to_parquet('/kaggle/working/train/train_applprev.parquet')

In [28]:
df_other = pd.read_csv("/kaggle/input/home-credit-credit-risk-model-stability/csv_files/train/train_other_1.csv")

In [29]:
df_other = reduce_mem_usage(df_other)

Memory usage of dataframe is 2.73 MB
Memory usage after optimization is: 1.22 MB
Decreased by 55.4%


In [30]:
df_other

,case_id,amtdebitincoming_4809443A,amtdebitoutgoing_4809440A,amtdepositbalance_4809441A,amtdepositincoming_4809444A,amtdepositoutgoing_4809442A,num_group1
0,43801,12466.600586,12291.200195,914.200012,0.0,304.800018,0
1,43991,3333.400146,3273.400146,0.000000,0.0,0.000000,0
2,44001,10000.000000,10000.000000,0.000000,0.0,0.000000,0
3,44053,0.000000,0.000000,2586.400146,0.0,88.800003,0
4,44130,63.799999,60.799999,0.000000,0.0,0.000000,0
...,...,...,...,...,...,...,...
51104,2703443,344.600006,343.399994,0.000000,0.0,0.000000,0
51105,2703448,0.000000,0.000000,2184.199951,0.0,18.000000,0
51106,2703450,0.000000,0.000000,0.000000,0.0,0.000000,0
51107,2703451,27500.000000,27477.599609,0.000000,0.0,0.000000,0
